使用特定初始化是让反向传播时，传播前和传播后，某层方差保持不变，从而缓解梯度爆炸和梯度消失的问题

普通的sigmoid函数的平均值为0.5并非0，导致神经元输出的方差将大于输入的方差


In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
epochs = 1

In [2]:
# Glorot 和 He 初始化
[name for name in dir(keras.initializers) if not name.startswith('_')]

['Constant',
 'GlorotNormal',
 'GlorotUniform',
 'HeNormal',
 'HeUniform',
 'Identity',
 'Initializer',
 'LecunNormal',
 'LecunUniform',
 'Ones',
 'Orthogonal',
 'RandomNormal',
 'RandomUniform',
 'TruncatedNormal',
 'VarianceScaling',
 'Zeros',
 'constant',
 'deserialize',
 'get',
 'glorot_normal',
 'glorot_uniform',
 'he_normal',
 'he_uniform',
 'identity',
 'lecun_normal',
 'lecun_uniform',
 'ones',
 'orthogonal',
 'random_normal',
 'random_uniform',
 'serialize',
 'truncated_normal',
 'variance_scaling',
 'zeros']

|初始化|激活函数|$\sigma^2$|
|-|-|-|
|Glorot|None,tanh,logistic,softmax|$\frac{1}{fan_{avg}}$|
|He|ReLU和变体|$\frac{2}{fan_{in}}$|
|LeCun|SELU|$\frac{1}{fan_{in}}$|

In [3]:
keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

In [4]:
# 如果使用均匀分布但基于fan_avg而不是fan_in
# 进行He初始化，则可以使用Variance_Scaling初始化
init = keras.initializers.VarianceScaling(scale=2, mode='fan_avg'
                                , distribution='uniform')
        
keras.layers.Dense(10, activation='relu', kernel_initializer=init)

相比于ReLU，leaky ReLU能够防止加权和为负数时，神经元只会输出0导致神经元死亡
$$ReLU(z)=\max(0,z) $$

$$leakyReLU(z)=\max(\alpha z,z) $$

后续的，还有随机泄露ReLU（RReLU），在训练过程中在给定范围内随机选择 $\alpha $

有参数化泄露ReLU（PReLU）， $\alpha $将在训练期间学习而不作为超参数

还有一种激活函数，指数线性单位（Exponential Linear Unit，ELU）

$$
ELU(z)=
\begin{cases}

\alpha(e^z - 1)\quad if\quad z<0\\
z\quad if\quad z\geq 0


\end{cases}
$$
如果 $\alpha=1 $则函数所有位置都是平滑的，有助于梯度加速下降

ELU的主要缺点时它的计算比ReLU及变体要慢

SELU 可以使得网路是自归一化的，但是有条件p298

In [5]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [6]:
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [7]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [8]:
history = model.fit(X_train, y_train, epochs=epochs,
                    validation_data=(X_valid, y_valid))

1719/1719 [==============================] - 11s 6ms/step - loss: 1.2897 - accuracy: 0.6043 - val_loss: 0.8785 - val_accuracy: 0.7236


* Batch Normaliztion批量归一化
  
    更换激活函数成功缓解了训练一开始时的梯度爆炸、消失，而无法保证训练一段时间后的旧疾复发

    批量归一化通过让模型学习各层输入的最佳缩放和均值，从而缓解训练一段时间后的梯度爆炸、消失
* 大致方法

    在隐藏层的激活函数前，通过该层输入的均值和标准差的 **移动平均值**来估计训练期间的最终统计信息

    1. 通过反向传播想学习输出缩放向量$\gamma $和输出偏移向量$\beta $
    2. 学习使用指数移动平均值估计的最终的输入均值向量$\mu $和最终输入标准差向量$\sigma $
* 具体实现

    使用API实现非常简单直观，只需在每个隐藏层的激活函数之前或之后添加一个批量归一化的层

    但是BN论文的作者主张在激活函数之前添加批量归一化层

    首先是在激活后使用BN，随后是在激活前使用BN

In [9]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    # 可选地在模型的第一层后添加一个BN
    keras.layers.BatchNormalization(), 
    # 在激活函数后添加BN 
    keras.layers.Dense(300, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation='softmax')
])

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense_5 (Dense)             (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_6 (Dense)             (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)             

In [11]:
# 每个BN层的每个输入添加了四个参数
bn1 = model.layers[1]
[(var.name, var.trainable) for var in bn1.variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [12]:
model.compile(loss='sparse_categorical_crossentropy',
            optimizer=keras.optimizers.SGD(learning_rate=1e-3),
            metrics=["accuracy"])

In [13]:
history = model.fit(X_train, y_train, epochs=epochs,
                    validation_data=(X_valid, y_valid))


1719/1719 [==============================] - 25s 14ms/step - loss: 0.8254 - accuracy: 0.7207 - val_loss: 0.5540 - val_accuracy: 0.8104


现在试试在激活之前使用BN层

In [14]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    # 可选地在模型的第一层之后添加一个BN层
    keras.layers.BatchNormalization(), 
    # 在激活函数之前使用BN， 
    # 在BN层之前的层不需要偏置项，因为BN自带偏置项
    keras.layers.Dense(300, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(100, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(10, activation='softmax')
])

In [15]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [16]:
history = model.fit(X_train, y_train, epochs=epochs,
                    validation_data=(X_valid, y_valid))

1719/1719 [==============================] - 24s 14ms/step - loss: 1.0333 - accuracy: 0.6750 - val_loss: 0.6803 - val_accuracy: 0.7852


在反向传播期间裁剪梯度，让它们永远不会超过某个阈值，是另一个缓解梯度爆炸的流行技术

keras中仅仅需要设置clipvalue或clipnorm参数，即可实现梯度裁剪

该优化器会将梯度向量的每个分量都蔡建伟-1.0到1.0之间的值，即将所有损失的偏导数限制在这个范围之间，阈值是可以调整的超参数

应当通过设置clipnorm而不是clipvalue按照范数裁剪，确保梯度裁剪不会改变梯度的方向（按值裁剪、按范数裁剪）

In [17]:
from pickletools import optimize


optimizer = keras.optimizers.SGD(clipvalue=1.0)
optimizer = keras.optimizers.SGD(clipnorm=1.0)

用keras进行迁移学习

In [18]:
# 分割fashion Mnist数据集，返回两个训练集
# A训练集 X-标签不是凉鞋且不是衬衫的所有图片，y-X的标签列（多分类器）
# B训练集 X-标签是凉鞋或衬衫的所有图片，y-X的标签是否为衬衫的标签列（二元分类器）
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))
# 按类别分离Fashion Mnist的三大集合
(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [19]:
X_train_A.shape

(43986, 28, 28)

In [20]:
X_train_B.shape

(200, 28, 28)

In [21]:
y_train_A[:30]

array([4, 0, 5, 7, 7, 7, 4, 4, 3, 4, 0, 1, 6, 3, 4, 3, 2, 6, 5, 3, 4, 5,
       1, 3, 4, 2, 0, 6, 7, 1], dtype=uint8)

In [22]:
y_train_B[:30]

array([1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1.], dtype=float32)

In [23]:
# 组件模型A，编译并训练
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [24]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [25]:
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
1375/1375 [==============================] - 14s 9ms/step - loss: 0.5515 - accuracy: 0.8223 - val_loss: 0.3899 - val_accuracy: 0.8637
Epoch 2/20
1375/1375 [==============================] - 14s 10ms/step - loss: 0.3530 - accuracy: 0.8794 - val_loss: 0.3299 - val_accuracy: 0.8847
Epoch 3/20
1375/1375 [==============================] - 14s 10ms/step - loss: 0.3148 - accuracy: 0.8909 - val_loss: 0.3002 - val_accuracy: 0.8946
Epoch 4/20
1375/1375 [==============================] - 13s 9ms/step - loss: 0.2950 - accuracy: 0.8981 - val_loss: 0.2853 - val_accuracy: 0.8996
Epoch 5/20
1375/1375 [==============================] - 13s 9ms/step - loss: 0.2819 - accuracy: 0.9025 - val_loss: 0.2768 - val_accuracy: 0.9023
Epoch 6/20
1375/1375 [==============================] - 13s 9ms/step - loss: 0.2723 - accuracy: 0.9070 - val_loss: 0.2727 - val_accuracy: 0.9051
Epoch 7/20
1375/1375 [==============================] - 13s 9ms/step - loss: 0.2642 - accuracy: 0.9090 - val_loss: 0.2685 - val_

In [26]:
model_A.save("my_model_A.h5")

In [36]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
# 参考模型A的架构并训练一个新模型 
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [37]:
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=['accuracy'])

In [38]:
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 [==============================] - 1s 82ms/step - loss: 0.6294 - accuracy: 0.6850 - val_loss: 0.5325 - val_accuracy: 0.7748
Epoch 2/20
7/7 [==============================] - 0s 47ms/step - loss: 0.4365 - accuracy: 0.8400 - val_loss: 0.4283 - val_accuracy: 0.8377
Epoch 3/20
7/7 [==============================] - 0s 50ms/step - loss: 0.3464 - accuracy: 0.8850 - val_loss: 0.3589 - val_accuracy: 0.8986
Epoch 4/20
7/7 [==============================] - 0s 51ms/step - loss: 0.2871 - accuracy: 0.9300 - val_loss: 0.3082 - val_accuracy: 0.9168
Epoch 5/20
7/7 [==============================] - 0s 49ms/step - loss: 0.2425 - accuracy: 0.9450 - val_loss: 0.2726 - val_accuracy: 0.9320
Epoch 6/20
7/7 [==============================] - 0s 48ms/step - loss: 0.2110 - accuracy: 0.9600 - val_loss: 0.2439 - val_accuracy: 0.9442
Epoch 7/20
7/7 [==============================] - 0s 46ms/step - loss: 0.1866 - accuracy: 0.9700 - val_loss: 0.2241 - val_accuracy: 0.9493
Epoch 8/20
7/7 [===========

In [39]:
model_B.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_7 (Flatten)         (None, 784)               0         
                                                                 
 dense_34 (Dense)            (None, 300)               235500    
                                                                 
 dense_35 (Dense)            (None, 100)               30100     
                                                                 
 dense_36 (Dense)            (None, 50)                5050      
                                                                 
 dense_37 (Dense)            (None, 50)                2550      
                                                                 
 dense_38 (Dense)            (None, 50)                2550      
                                                                 
 dense_39 (Dense)            (None, 1)                

任务B与任务A非常相似，上面的模型B通过模仿模型A的架构来完成训练

现在也可以通过迁移学习来完成

In [42]:
model_A = keras.models.load_model("my_model_A.h5")
# 获取除了输出层之前的所有层架构及其训练好的权重
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

model_A.layers[:-1]

现在model_A和model_B_on_A共享一些层，对后者的训练也会影响前者的层权重

如果想要分离控制二者，则必须使用api，clone_model()

In [43]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())
model_B_on_A = keras.models.Sequential(model_A_clone.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

由于新的输出层是随机初始化的，可能存在较大的错误梯度，可能会破坏重用模型A的现成权重

* 冻结训练

    为了避免以上情况，就是在训练的前几个伦茨冻结重用的层，给新层一些时间来学习合理的权重
* 代码

    将每一层的可训练属性设置为False并编译模型

In [45]:
# 冻结重用的前几层（此处是除了是输出层以外的所有层）
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss='binary_crossentropy',
                    optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                    metrics=["accuracy"])

In [47]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                            validation_data=(X_valid_B, y_valid_B))
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                    optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                    metrics=["accuracy"])

history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                            validation_data=(X_valid_B, y_valid_B))



Epoch 1/4
7/7 [==============================] - 0s 54ms/step - loss: 0.6020 - accuracy: 0.7000 - val_loss: 0.5433 - val_accuracy: 0.7525
Epoch 2/4
7/7 [==============================] - 0s 45ms/step - loss: 0.5722 - accuracy: 0.7300 - val_loss: 0.5162 - val_accuracy: 0.7708
Epoch 3/4
7/7 [==============================] - 0s 46ms/step - loss: 0.5430 - accuracy: 0.7400 - val_loss: 0.4927 - val_accuracy: 0.7819
Epoch 4/4
7/7 [==============================] - 0s 44ms/step - loss: 0.5174 - accuracy: 0.7500 - val_loss: 0.4698 - val_accuracy: 0.7972
Epoch 1/16
7/7 [==============================] - 1s 78ms/step - loss: 0.4317 - accuracy: 0.8050 - val_loss: 0.3198 - val_accuracy: 0.8742
Epoch 2/16
7/7 [==============================] - 0s 45ms/step - loss: 0.2911 - accuracy: 0.8950 - val_loss: 0.2389 - val_accuracy: 0.9168
Epoch 3/16
7/7 [==============================] - 0s 68ms/step - loss: 0.2141 - accuracy: 0.9400 - val_loss: 0.1927 - val_accuracy: 0.9422
Epoch 4/16
7/7 [===============

In [48]:
model_B.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 7ms/step - loss: 0.0993 - accuracy: 0.9865


[0.09932302683591843, 0.9865000247955322]

In [49]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 7ms/step - loss: 0.0606 - accuracy: 0.9915


[0.060557588934898376, 0.9915000200271606]

In [50]:
(100 - 98.65) / (100 - 99.15)

1.588235294117651

测试的精度高达98.58%, 错误率也降了1.59倍

这么做实际上是“折磨数据直到信服位置”

当论文看起来过于优秀时，你应该要怀疑：也许这个浮华的新技术实际上并没有多大的帮助（事实上，它甚至可能降低性能），但作者尝试了许多变体，仅报告了最好的结果（这可能是由于运气所致），而没有提及他们在途中遇到了多少次失败。

在大多数情况下，这根本不是恶意的，但这是造成如此多科学结果永远无法复现的部分原因

事实上，迁移学习在小型密集网络上不能很好地工作，可能因为小型网络学习的模式很少，密集网络学习的是非常特定的模式，在其他任务中不是很有用

迁移学习最适合深度卷积神经网络，该神经网络倾向于学习更为通用的特征检测器

更快的优化器

<font color='red'>公式中所有更新的变量均为向量</font>
* 动量优化

    动量优化非常关心先前的梯度（加速度概念），每次迭代都会从动量向量m中减去局部梯度，并通过添加该动量向量来更新权重

    为了模拟摩擦机制并防止动量变得过大，引入超参数$\beta $，另外$\eta $还是学习率

    $$m\leftarrow\beta m-\eta\nabla_{\theta}J(\theta) $$
    $$\theta\leftarrow\theta+m $$
* Nesterov加速梯度

    又称NNesterov动量优化，是动量优化的小变体，它不再局部位置$\theta $，而在$\theta+\beta m处沿动量方向稍微提前处测量成本函数的梯度 $

    $$m\leftarrow\beta m-\eta\nabla_{\theta}J(\theta+\beta m) $$
    $$\theta\leftarrow\theta+m $$

    这种小的调整有效是因为通常动量向量会指向正确的方向，因此使用在该方向上测得的更远的梯度而不是原始位置上的梯度会稍微准确一些

* AdaGrad

    通过沿着最陡峭的维度按比例缩小梯度向量

    此处是逐个元素相乘，相除
    $$s\leftarrow s+ \nabla_{\theta}J(\theta)\times\nabla_{\theta}J(\theta)$$
    $$\theta\leftarrow\theta-\eta\nabla_{\theta}J(\theta) / \sqrt{s+\epsilon} $$

    1. 将梯度的平方累计到向量s中
    2. 梯度向量按比例因子$\sqrt{s+\epsilon} $缩小了，其中$\epsilon是平滑项 $
    
    *该算法会降低学习率*，但是对于陡峭的维度，它的执行速度比对缓慢下降的维度的执行速度要快。这称为自适应学习率。好处是算法几乎不需要调整学习率超参数$\eta$
* RMSProp
    
    AdaGrad有可能下降太快，从而永远不能收敛到全局最优解。RMSProp解决了这个问题。

    其通过只是累加最近迭代中的梯度（而不是自训练依赖的所有梯度）来解决这个问题

    $$s\leftarrow \beta s+ (1-\beta)\nabla_{\theta}J(\theta)\times\nabla_{\theta}J(\theta)$$
    $$\theta\leftarrow\theta-\eta\nabla_{\theta}J(\theta) / \sqrt{s+\epsilon} $$

    衰减率$\beta$是一个新的超参数，设置为0.9,通过施加衰减率相关的加权和，强调了最近迭代的梯度，弱化了梯度平方的权重从而避免像AdaGrad那样下降太快

* Adam

    结合了动量优化和RMSProp的思想，代表着自适应矩估计,(逐个元素相乘、逐个元素相除)
    
    1. $$m\leftarrow\beta_1 m-(1-\beta_1)\nabla_{\theta}J(\theta) $$ 
    
    2. $$s\leftarrow \beta_2s+ (1-\beta_2)\nabla_{\theta}J(\theta)\times\nabla_{\theta}J(\theta)$$
    3. $$\hat m\leftarrow\frac{m}{1-\beta^t_1} $$
    4. $$\hat s\leftarrow\frac{s}{1-\beta_2^t} $$
    5. $$\theta\leftarrow\theta-\eta\hat m / \sqrt{s+\epsilon} $$

    1、2、5与动量优化和RMSProp非常相似
    3、4由于m和s初始化为0，因此在训练开始时它们会偏向0，这两个个步骤有助于在训练开始时提高m和s

    Adam也是一种自适应学习率算法，因此对学习率超参数$\eta$需要较少的调整

* Nadam
  
    是在Adam优化上加上了Nesterov技巧，因此其收敛速度通常比Adam快

**优化器比较P317**

**训练稀疏模型**

在训练时使用强l1正规化，迫使优化器产生尽可能多的为0的权重

In [51]:
# 实现动量优化, 0.9是一个动量可参考的超参值
from re import T


optimizer = keras.optimizers.SGD(learning_rate=1e-3, momentum=0.9)
# 实现Nesterov动量优化
optimizer = keras.optimizers.SGD(learning_rate=1e-3, momentum=0.9, nesterov=True)
# 不应该用AdaGrad
optimizer = keras.optimizers.Adagrad(learning_rate=0.001)
# 使用RMSProp
optimizer = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)
# 使用Adam beta_1和beta_2的参数是实用的，并非随意给出
optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
# 使用Nadam
optimizer = keras.optimizers.Nadam(learning_rate=1e-3, beta_1=0.9, beta_2=0.999)

学习率调度P318
* 幂调度

    $$\eta(t)=\frac{\eta_0}{(1+\frac{t}{s})^c} $$
    $$\eta_0是初始学习率 \quad t是迭代次数$$
    $$s是每s步学习率下降一次，第i次下降将下降到\frac{\eta_0}{i} $$
* 指数调度
    $$\eta(t)= \eta_0 0.1^{\frac{t}{s}}$$
* 分段恒定调度
* 性能调度
* 1周期调度

In [52]:
# 使用幂调度 decay参数是s的倒数
optimizer = keras.optimizers.SGD(learning_rate=0.01, decay=1e-4)

In [53]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])